Run below for the FULL feature engineered file with SOFA, NEWS, qSOFA, temporal sliding windows. If you're wanting feature reduction you need to also run the ones below this. 

In [14]:
from pathlib import Path

def find_project_root(marker=".gitignore"):
    """
    walk up from the current working directory until a directory containing the
    specified marker (e.g., .gitignore) is found.
    """
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(f"Project root marker '{marker}' not found starting from {current}")
  

## Pre-processing

In [17]:
root = find_project_root()
INPUT_DATASET = f"{root}/dataset/Fully_imputed_dataset.parquet"
OUTPUT_DATASET = f"{root}/dataset/preprocessed_data.parquet"
BALANCED_INPUT = f"{root}/dataset/feature_engineering/balanced_dataset.parquet"



def create_balanced_dataset(
    input_file=INPUT_DATASET, output_file="balanced_dataset.parquet"):
    root = find_project_root()
    data_path = root / "dataset" / input_file
    df = pd.read_parquet(data_path)

    if not isinstance(df.index, pd.MultiIndex):
        df.set_index(["patient_id", "ICULOS"], inplace=True)

    grouped = df.reset_index().groupby("patient_id")["SepsisLabel"]
    patient_is_positive = grouped.max()

    positive_ids = patient_is_positive[patient_is_positive == 1].index
    negative_ids = patient_is_positive[patient_is_positive == 0].index

    print(f"Number of positive patients: {len(positive_ids)}")
    print(f"Number of negative patients: {len(negative_ids)}")

    np.random.seed(1)
    sampled_negative_ids = np.random.choice( negative_ids, size=int(len(positive_ids) * (70 / 30)), replace=False)
    keep_ids = set(positive_ids).union(set(sampled_negative_ids))
    df_balanced = df.loc[df.index.get_level_values("patient_id").isin(keep_ids)]

    out_path = root / "dataset" / "feature_engineering"/output_file
    df_balanced.to_parquet(out_path)
    print(f"save to: {out_path}")

    final_label_map = (
        df_balanced.reset_index().groupby("patient_id")["SepsisLabel"].max()
    )

    final_positive = (final_label_map == 1).sum()
    final_negative = (final_label_map == 0).sum()

    print(f"retained positive patients: {final_positive}")
    print(f"retained negative patients: {final_negative}")
    print(f"Positive ratio: {final_positive / (final_positive + final_negative)}")

create_balanced_dataset()

Number of positive patients: 2932
Number of negative patients: 37404
save to: C:\Users\Administrator\Desktop\ds\dl-sepsis-prediction\dataset\feature_engineering\balanced_dataset.parquet
retained positive patients: 2932
retained negative patients: 6841
Positive ratio: 0.3000102322725877


In [ ]:
%pip install swifter

In [ ]:

import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import swifter

root = find_project_root()
INPUT_DATASET = f"{root}/dataset/Fully_imputed_dataset.parquet"
OUTPUT_DATASET = f"{root}/dataset/preprocessed_data.parquet"
BALANCED_INPUT = f"{root}/dataset/feature_engineering/balanced_dataset.parquet"

    
df = pd.read_parquet(BALANCED_INPUT)
df.head()
max_length = df.groupby("patient_id").size().max()

# Score Calculation
def calculate_sofa(row):
    sofa = 0

    def assign_score(value, thresholds):
        for threshold, score in thresholds:
            if value >= threshold:
                return score
        return 0  

    # Respiration 
    if row.get('FiO2', 0) > 0:
        pao2_fio2 = row.get('SaO2', 0) / row['FiO2']
        sofa += assign_score(pao2_fio2, [(100, 4), (200, 3), (300, 2), (400, 1)])

    # Coagulation
    sofa += assign_score(row.get('Platelets', float('inf')), [(20, 4), (50, 3), (100, 2), (150, 1)])

    # Liver Function
    sofa += assign_score(row.get('Bilirubin_total', 0), [(12, 4), (6, 3), (2, 2), (1.2, 1)])

    # Cardiovascular
    if row.get('MAP', 100) < 70:
        sofa += 1

    # Renal Function
    sofa += assign_score(row.get('Creatinine', 0), [(5, 4), (3.5, 3), (2, 2), (1.2, 1)])

    return sofa

def calculate_news(row):
    news = 0

    def assign_news_score(value, thresholds):
        for threshold, score in thresholds:
            if value >= threshold:
                return score
        return 0  

    # HR (Heart Rate)
    news += assign_news_score(row.get('HR', 0), [(40, 3), (50, 1), (90, 0), (110, 1), (130, 2), (131, 3)])

    # Respiration Rate
    news += assign_news_score(row.get('Resp', 0), [(8, 3), (9, 1), (11, 0), (21, 2), (24, 3)])

    # Temperature
    news += assign_news_score(row.get('Temp', 0), [(35, 3), (36, 1), (38, 1), (39.1, 2)])

    # SBP (Systolic BP) or MAP (Mean Arterial Pressure)
    sbp = row.get('SBP', row.get('MAP', 100))
    news += assign_news_score(sbp, [(90, 3), (100, 2), (110, 1)])

    # O2 Saturation
    news += assign_news_score(row.get('O2Sat', 0), [(85, 3), (91, 2), (93, 1)])

    # Supplemental Oxygen (if available)
    if row.get('FiO2', 0) > 0.21:
        news += 2

    return news

def calculate_qsofa(row):
    qsofa = 0

    # SBP ≤ 100 mmHg
    if row.get('SBP', 120) <= 100:
        qsofa += 1

    # Respiration Rate ≥ 22
    if row.get('Resp', 0) >= 22:
        qsofa += 1

    return qsofa

def num_recorded_values(row):
    recorded_measurements = df.notnull().sum()

    return recorded_measurements

def missingness_feature(row):
    
    if 'ICULOS' and 'ICULOS' in df.columns:
        df = df.sort_values(by='ICULOS')
        time_intervals = df['ICULOS'].diff()

    return time_intervals


def add_temporal_features(df):
    time_window_sizes = [3,6,12]
    feature_cols = ["HR", "Resp", "Temp"]

    df.sort_values(['patient_id', 'ICULOS'], inplace=True)

    for col in feature_cols:
        if col in df.columns:
            for window in time_window_sizes:
                rolling = df.groupby('patient_id')[col].transform(lambda x: x.rolling(window, min_periods=1))
                df[f'{col}_mean_{window}h'] = rolling.mean()
                df[f'{col}_max_{window}h'] = rolling.max()
                df[f'{col}_last_{window}h'] = rolling.apply(lambda x: x.iloc[-1] if len(x) > 0 else np.nan, raw=False)

    rolling_cols = [col for col in df.columns if any(stat in col for stat in ["_mean_", "_max_", "_last_"])]
    df[rolling_cols] = df[rolling_cols].fillna(0)
    df = df[df['ICULOS'] >= 11]

    return df


def preprocess_data(output_file):
    global df 

    df['SOFA'] = df.swifter.apply(calculate_sofa, axis=1)
    df['NEWS'] = df.swifter.apply(calculate_news, axis=1)
    df['qSOFA'] = df.swifter.apply(calculate_qsofa, axis=1)
    #df['num_recorded_values'] = df.apply(num_recorded_values, axis=1)
    #df['missingness_feature'] = df.apply(missingness_feature, axis=1)
    df = add_temporal_features(df)
    
    
    # drop columns that are not needed
    columns_to_drop = ["Unit1", "Unit2", "cluster_id", "dataset", "HospAdmTime"]
    df = df.drop(columns=columns_to_drop)
    
    # NOTE: handle nan values doing forward fill and then back fill
    df = df.fillna(method="ffill")
    df = df.fillna(method="bfill")

    df.to_parquet(output_file, index=False)

    print(f"Preprocessed data saved to {output_file}")
    sample_patient_id = df['patient_id'].unique()[0]
    preview = df[df['patient_id'] == sample_patient_id]
    preview_path = str(output_file).replace(".parquet", "_preview.csv")
    preview.to_csv(preview_path, index=False)
    print(f"Preview of patient saved to: {preview_path}")


preprocess_data(OUTPUT_DATASET)

Note: you may need to restart the kernel to use updated packages.


Pandas Apply: 100%|██████████| 424368/424368 [00:03<00:00, 127677.33it/s]


KeyboardInterrupt: 

## PCA
If you're wanting the PCA analysis dataset run the code below to save it as pca_preprocessed.parquet in YOUR files (Github says file too large now)

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

root = find_project_root()
INPUT_DATASET = f"{root}/dataset/preprocessed_data.parquet"
OUTPUT_DATASET = f"{root}/dataset/pca_preprocessed_data.parquet"

df = pd.read_parquet(INPUT_DATASET)

keep_columns = ["SepsisLabel", "patient_id"]
stored_columns = df[keep_columns].copy()

# PCA process
X = df.drop(columns=keep_columns)
print(X.var())
print(X.dtypes)
y = df["SepsisLabel"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

print(f"Original Features: {X.shape[1]}, Reduced Features: {X_pca.shape[1]}")
print(X_pca)

# Plot
explained_variance_ratio = pca.explained_variance_ratio_
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(explained_variance_ratio), marker="o", linestyle="--")
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("PCA Explained Variance")
plt.grid()
plt.show()

# Create final DataFrame with all columns
X_pca_df = pd.DataFrame(X_pca)
# Add back the stored columns
for col in keep_columns:
    X_pca_df[col] = stored_columns[col]

# Save complete DataFrame
X_pca_df.to_parquet(OUTPUT_DATASET, index=False)

## Feature selection
Run below for feature selection, this is based on SHAP values + preliminary model training to see which features were most impactful + what is the optimal number of features

In [ ]:

file_path = "preprocessed_data.parquet" 
df = pd.read_parquet(file_path)

X = df.drop(columns=["SepsisLabel", "patient_id", "dataset", "cluster_id", "Unit2", "Gender", "Platelets_Delta", "Creatinine_Delta", "Platelets_SD_3h", "Hct", "MAP_Delta", "MAP_SD_3h", "Creatinine_MA_3h", "Creatinine_SD_3h", "Platelets"])
X = X.fillna(method="bfill")
X.to_parquet("feature_selection_preprocessed.parquet", index=False)